# Day-ahead Price Exploratory Data Analysis

## Concatenating years 2019-2022

In [41]:
import pandas as pd
import numpy as np 
import matplotlib as plt

# concat year files to one dataframe
dayahead = pd.DataFrame(pd.read_csv("day-ahead-prices/Day-ahead_Prices_201901010000-202001010000.csv", header=0))
dayahead = pd.concat([dayahead, pd.read_csv("day-ahead-prices/Day-ahead_Prices_202001010000-202101010000.csv", header=0)])
dayahead = pd.concat([dayahead, pd.read_csv("day-ahead-prices/Day-ahead_Prices_202101010000-202201010000.csv", header=0)])
dayahead = pd.concat([dayahead, pd.read_csv("day-ahead-prices/Day-ahead_Prices_202201010000-202301010000.csv", header=0)])

# create datetime value of every beginning hour
dayahead['datetime'] = dayahead['MTU (UTC)'].str[0:16]
dayahead['datetime'] = pd.to_datetime(dayahead['datetime'], format='%d.%m.%Y %H:%M')
dayahead.set_index('datetime', inplace=True)

#do some cleanup
dayahead.rename(columns = {'Day-ahead Price [EUR/MWh]':'price'}, inplace = True)
dayahead.drop(columns={'BZN|FI', 'MTU (UTC)', 'Currency'}, inplace=True)
dayahead['price'] = dayahead['price'].replace('-', np.nan).astype(float)
np.shape(dayahead)


(35064, 1)

## Look for data tail

In [58]:
print(dayahead.head())
print(dayahead.tail())

                     price
datetime                  
2019-01-01 00:00:00  10.07
2019-01-01 01:00:00  10.03
2019-01-01 02:00:00   4.56
2019-01-01 03:00:00   4.83
2019-01-01 04:00:00   8.09
                     price
datetime                  
2022-12-31 19:00:00    NaN
2022-12-31 20:00:00    NaN
2022-12-31 21:00:00    NaN
2022-12-31 22:00:00    NaN
2022-12-31 23:00:00    NaN


## Find last non-NaN value

In [59]:
print(dayahead.iloc[1500])
print(dayahead.iloc[2000])
print(dayahead.iloc[1970])

price    52.41
Name: 2019-03-04 12:00:00, dtype: float64
price    41.9
Name: 2019-03-25 08:00:00, dtype: float64
price    32.2
Name: 2019-03-24 02:00:00, dtype: float64


## Count missing values

In [64]:
missing_sum = dayahead[dayahead.index < '2022-10-06'].isna().sum()
print(missing_sum / dayahead[dayahead.index < '2022-10-06'].count())

price    0.003713
dtype: float64


## Impute missing

Since missing values are quite few (less than 0.4 %), they can be imputed with values next to then (which quite clearly is smarter than imputing them with mean values).

In [67]:
dayahead.fillna(method='ffill', inplace=True)
print(dayahead[dayahead.index < '2022-10-06'].isna().sum())

price    0
dtype: int64


## Export joined day-ahead data as parquet file

Change price column name to be more simple and datatype to float

In [68]:
import pyarrow as pa

print(np.dtype(dayahead['price']))

table = pa.Table.from_pandas(dayahead)
pa.parquet.write_table(table, 'processed_data/dayahead.parquet')


float64
